In [26]:
import os


In [27]:
%pwd

'/Users/rociocuatecontzi/Desktop/programming/electricity-forecast'

In [3]:
os.chdir("../")

In [34]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    csv_name: Path

In [35]:
from src.eForecaster.constants import *
from eForecaster.utils.common import read_yaml, create_directories

In [49]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            csv_name=config.csv_name 
        )

        return data_ingestion_config
      

In [50]:
import os
import urllib.request as request
import zipfile
from eForecaster import logger
from eForecaster.utils.common import get_size
import pandas as pd

In [77]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    
    def split_dataset(self):
        path_to_csv = os.path.join(self.config.unzip_dir, self.config.csv_name)
        df = pd.read_csv(path_to_csv, sep=';', decimal=',')
        before_symbol = df['datetime'].str.split('+').str[0]
        df["datetime"] = pd.to_datetime(before_symbol, format="%Y.%m.%d %H:%M:%S ")
        
        df.set_index('datetime', inplace=True)
        print(df.index.hour)
        df['hour'] = df.index.hour
        df['dayofweek'] = df.index.dayofweek
        df['quarter'] = df.index.quarter
        df['month'] = df.index.month
        df['year'] = df.index.year
        df['dayofyear'] = df.index.dayofyear
        df['minute'] = df.index.minute
        start_date = '2021-10-04'
        train=df[(df.index<start_date)]
        test=df[(df.index>=start_date)]

        unzip_path = self.config.unzip_dir
        train_csv_name = Path("train.csv")
        test_csv_name = Path("test.csv")
        train.to_csv(path_or_buf=os.path.join(unzip_path,train_csv_name))
        test.to_csv(path_or_buf=os.path.join(unzip_path,test_csv_name))

In [78]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    data_ingestion.split_dataset()
except Exception as e:
    raise e

[2024-01-26 19:39:40,802: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-26 19:39:40,805: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-26 19:39:40,806: INFO: common: created directory at: artifacts]
[2024-01-26 19:39:40,807: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-26 19:39:40,809: INFO: 3163113933: File already exists of size: ~ 2532 KB]
Index([ 0,  0,  0,  1,  1,  1,  1,  2,  2,  2,
       ...
       21, 22, 22, 22, 22, 23, 23, 23, 23,  0],
      dtype='int32', name='datetime', length=377084)
                        power  hour  dayofweek  quarter  month  year  \
datetime                                                               
2012-01-01 00:15:00  3767.843     0          6        1      1  2012   
2012-01-01 00:30:00  3743.281     0          6        1      1  2012   
2012-01-01 00:45:00  3691.766     0          6        1      1  2012   
2012-01-01 01:00:00  3645.126     1          6        1      